In [96]:
import os
import time
import numpy as np
import openai
from openai import AzureOpenAI
from importlib import reload
import sys
import pprint

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.ai.documentintelligence import DocumentIntelligenceClient

file_name="./uploads/Sample_Invoice.pdf"

### Get the ADI document parsed using Azure document analyzer.
Also return the propmpt parameters, message to pass for open ai chat completion

In [74]:
def get_azure_parsed_contents(file_data):

    adi_end_point=os.getenv("AZURE_ENDPOINT")
    adi_api_key=os.getenv("AZURE_KEY")

    document_analysis_client=DocumentIntelligenceClient(
            endpoint=adi_end_point, 
            credential=AzureKeyCredential(adi_api_key)
            )
    
    poller=document_analysis_client.begin_analyze_document("prebuilt-invoice",
                analyze_request=file_data,
                content_type="application/octet-stream"
                )
    
    result = poller.result()

    return result

In [75]:
st_time=time.time()
file_data = open(file_name, "rb")
adi_response=get_azure_parsed_contents(file_data)
en_time=time.time()
print(f"Time elapsed: {np.round(en_time-st_time,2)} seconds.")

Time elapsed: 13.53 seconds.


In [76]:
print(type(adi_response))

<class 'azure.ai.documentintelligence.models._models.AnalyzeResult'>


In [77]:
print(adi_response.keys())

dict_keys(['apiVersion', 'modelId', 'stringIndexType', 'content', 'pages', 'tables', 'styles', 'documents', 'contentFormat'])


In [109]:
# print(adi_response['modelId'])

# adi_response['content']

# adi_response['apiVersion']

# doc=adi_response['documents'][0]

# doc.keys()

# adi_response['pages'][0].keys()

# len(adi_response['pages'][0]['words'])

# adi_response['pages'][0]['words'][:2]

# len(adi_response['pages'][0]['lines'])

# adi_response['pages'][0]['lines'][:2]

# adi_response['pages'][0]['spans']

# adi_response['pages'][0]['width']

# adi_response['pages'][0]['height']

# adi_response['pages'][0].keys()

# adi_response['pages'][0]['angle']

# adi_response['pages'][0]['unit']

In [110]:
for page in adi_response.pages:
    print(page)

{'pageNumber': 1, 'angle': 0, 'width': 8.5, 'height': 11, 'unit': 'inch', 'words': [{'content': 'Spa', 'polygon': [2.6025, 1.0169, 3.6292, 1.0026, 3.6388, 1.6185, 2.6121, 1.6328], 'confidence': 0.974, 'span': {'offset': 0, 'length': 3}}, {'content': 'Supplies', 'polygon': [2.6216, 1.7617, 4.9042, 1.7331, 4.9376, 2.3633, 2.6216, 2.3633], 'confidence': 0.993, 'span': {'offset': 4, 'length': 8}}, {'content': '68', 'polygon': [1.3801, 2.7023, 1.5376, 2.6975, 1.5376, 2.8407, 1.3848, 2.8407], 'confidence': 0.996, 'span': {'offset': 13, 'length': 2}}, {'content': 'Vroom', 'polygon': [1.5711, 2.6975, 1.934, 2.6975, 1.934, 2.8407, 1.5758, 2.8407], 'confidence': 0.995, 'span': {'offset': 16, 'length': 5}}, {'content': 'Street,', 'polygon': [2.0343, 2.6975, 2.4354, 2.6975, 2.4354, 2.8407, 2.039, 2.8407], 'confidence': 0.993, 'span': {'offset': 22, 'length': 7}}, {'content': 'Medalia', 'polygon': [1.3801, 2.9266, 1.8767, 2.9362, 1.8719, 3.089, 1.3801, 3.0794], 'confidence': 0.994, 'span': {'offset

In [125]:
page=adi_response.pages[0]
lines=page['lines']

In [113]:
page.keys()

dict_keys(['pageNumber', 'angle', 'width', 'height', 'unit', 'words', 'lines', 'spans'])

In [116]:
# for line in page['lines']:
    # 

In [117]:
line=page['lines'][0]

In [120]:
print(line)

{'content': 'Spa', 'polygon': [2.4927, 1.0217, 3.6913, 1.0026, 3.7056, 1.6185, 2.507, 1.6328], 'spans': [{'offset': 0, 'length': 3}]}


In [119]:
line.content

'Spa'

In [121]:
page['lines'][1]

{'content': 'Supplies', 'polygon': [2.6169, 1.7378, 4.9472, 1.7283, 4.9472, 2.3537, 2.6169, 2.3633], 'spans': [{'offset': 4, 'length': 8}]}

In [129]:
content=""

for line in lines:
    content+="\n"+line['content']

In [130]:
content

'\nSpa\nSupplies\n68 Vroom Street,\nMedalia City,\nMM 88531\nPhone: 222-222-9000\nFax:\n222-222-9001\nINVOICE\nEmail: tonsofcars@it.com\nBill To:\nOlympic World Hotel.\n55 Comfort Street\nMedalia City,\nMM 88531\nInvoice #:\nPLC0920\nInvoice Date:\nMarch 2,20X3\nCustomer ID:\n124\nVendor No:\n15\nTO number:\nGSQ0415DB0037\nQuantity\nItems\nUnits\nDescri\nDiscount %\nTaxable\nUnit Price\nTotal\n14\nMTBL\n1\nMassage tables\nN/A\nN/A\n$2,300\n$32,200\n-\n14\nTH\n1\nTowel heaters\n-\nN/A\nN/A\n$768\n$10,752\n7\nEWCH\nI\n1\nEntry-way chairs\nN/A\nN/A\n$1,136\n$7,952\n14\nWAX\n1\nWax heaters\nN/A\nN/A\n$987\n$13,818\n16\nSLOU\n1\nSpa loungers\nN/A!\nN/A\n$979\n$15,616\n4\nHTUB\n1\nHot tub\nN/A\nN/A\n$50,613\n$202,453\n12\nPEDC\nI\n1\nPedicure chair\nN/A\nN/A\n$3,456\n$41,472\n4\nDRY\nI\n2\nManicure and Pedicure dryer\nN/A\nN/A\n$1,456\n$5,824\n10\nMANI\n1\n1\nManicure tables\nN/A\nI\nN/A ;\n$2,488\n$24,880\n-\n-\nI\nI\n-\nTotal\n$354,967\nDISCLAIMER: The information contained herein is of a 